In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Find all processed files in the input directory
processed_files = glob.glob(os.path.join(input_dir, "*_hole_projection.tif"))

# If no specific files found, look for any TIF files
if len(processed_files) == 0:
    processed_files = glob.glob(os.path.join(input_dir, "*.tif"))

print(f"Found {len(processed_files)} TIF files to process")
if len(processed_files) == 0:
    print("No TIF files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Function to find threshold using valley-based method
def find_valley_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=4)

    # Find valleys (local minima) in the smoothed histogram
    # We invert the histogram because find_peaks finds maxima
    valleys, _ = find_peaks(-hist_smooth)

    # Find valleys to the left of the peak
    valleys_left = valleys[valleys < peak_idx]

    # If there are valleys to the left, find the closest one to the peak
    if len(valleys_left) > 0:
        first_valley_idx = valleys_left[-1]  # Last valley to the left of the peak
        valley_value = bins[first_valley_idx]
        threshold = valley_value
        print(f"First valley found at intensity value: {valley_value:.4f}")
    else:
        # If no valleys are found, set threshold to 0
        threshold = 0
        print("No clear valley found. Setting threshold to 0")

    return threshold, hist, hist_smooth, bins, peak_idx, valleys_left

# Process each file
for file_path in processed_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Find threshold using the valley-based method
    threshold, hist, hist_smooth, bins, peak_idx, valleys_left = find_valley_threshold(illumination_corrected)

    # Apply thresholding to segment the image
    segmented = np.zeros_like(illumination_corrected)
    segmented[illumination_corrected <= threshold] = 1  # Segment pixels with intensity <= threshold

    # Visualization
    plt.figure(figsize=(15, 10))

    # Plot the original histogram with the detected peak and valley
    plt.subplot(2, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    if len(valleys_left) > 0:
        for valley_idx in valleys_left:
            plt.axvline(x=bins[valley_idx], color='y', linestyle=':', alpha=0.5)
    plt.title('Intensity Histogram with Detected Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Display the original image
    plt.subplot(2, 2, 2)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image')

    # Display the segmented image
    plt.subplot(2, 2, 3)
    plt.imshow(segmented, cmap='binary')
    plt.title('Segmented Image')

    # Show a comparison of the original and segmented images
    plt.subplot(2, 2, 4)
    # Create an RGB image for overlay
    overlay = np.zeros((*illumination_corrected.shape, 3))
    # Original in grayscale
    for i in range(3):
        overlay[..., i] = illumination_corrected
    # Segmented areas in red
    overlay[segmented == 1, 0] = 1  # Red channel
    overlay[segmented == 1, 1] = 0  # Green channel
    overlay[segmented == 1, 2] = 0  # Blue channel
    plt.imshow(overlay)
    plt.title('Overlay: Segmented Areas in Red')

    plt.tight_layout()

    # Save the visualization to the images directory
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path}")
    plt.close()

    # REMOVED: No longer saving illumination corrected image

    # Save ONLY the segmented image to the TIFF directory
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    print(f"  Saved segmented image to: {seg_path}")

print("\nAll processing completed!")

Mounted at /content/drive
Found 24 TIF files to process

Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1).tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
Peak found at intensity value: 0.0586
No clear valley found. Setting threshold to 0
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes_images/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1)_visualization.png
  Saved segmented image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1)_segmented.tif

Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional.tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gau

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage
from scipy.signal import find_peaks, savgol_filter
import os
import glob
from google.colab import drive

# Mount Google Drive if using Colab
try:
    drive.mount('/content/drive')
    USING_COLAB = True
except:
    USING_COLAB = False
    print("Not running in Colab, skipping drive mount")

# Define the input and output paths
# Updated to match your directory structure
if USING_COLAB:
    input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-x20/Cadherins/background'
    output_tif_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes'
    output_img_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes_images'
else:
    # Local paths (change these as needed)
    input_dir = './'
    output_tif_dir = './segmented/tiffs'
    output_img_dir = './segmented/images'

# Create output directories if they don't exist
os.makedirs(output_tif_dir, exist_ok=True)
os.makedirs(output_img_dir, exist_ok=True)

# Find all processed files in the input directory
processed_files = glob.glob(os.path.join(input_dir, "*_hole_projection.tif"))

# If no specific files found, look for any TIF files
if len(processed_files) == 0:
    processed_files = glob.glob(os.path.join(input_dir, "*.tif"))

print(f"Found {len(processed_files)} TIF files to process")
if len(processed_files) == 0:
    print("No TIF files found. Please check the input directory.")
    # Option to look for other file types
    all_files = glob.glob(os.path.join(input_dir, "*"))
    if len(all_files) > 0:
        print(f"Found {len(all_files)} files of other types in the directory.")
        print("Available files:")
        for file in all_files:
            print(f" - {os.path.basename(file)}")

# Function to apply illumination correction
def apply_illumination_correction(img):
    # Normalize image to [0, 1] range for processing
    img_min = np.min(img)
    img_max = np.max(img)
    if img_max > img_min:  # Check to prevent division by zero
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = img.astype(float)
        print("  Warning: Image has constant values, normalization skipped")

    # Estimate background using a large Gaussian filter
    print("Applying illumination correction via division method...")
    sigma = max(img_norm.shape) // 20  # Divisor of 20 for division method
    print(f"  Using Gaussian filter with sigma = {sigma}")

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Modified function to find threshold using first derivative minimum
def find_derivative_threshold(img):
    # Create a histogram of intensity values
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))

    # Find the peak intensity (mode)
    peak_idx = np.argmax(hist)
    peak_value = bins[peak_idx]
    print(f"Peak found at intensity value: {peak_value:.4f}")

    # Apply a smoothing filter to the histogram to reduce noise
    hist_smooth = savgol_filter(hist, window_length=11, polyorder=4)

    # Calculate the first derivative of the smoothed histogram
    derivative = np.diff(hist_smooth)

    # Find minimum derivative point to the left of the peak
    # This corresponds to the steepest downward slope
    if peak_idx > 1:  # Make sure we have points to the left of the peak
        # Only consider part of the derivative to the left of the peak
        left_derivative = derivative[:peak_idx-1]

        # Find the minimum derivative point (steepest downward slope)
        min_deriv_idx = np.argmin(left_derivative)

        # The threshold corresponds to the bin value at this point
        # +1 because derivative has one fewer element than hist_smooth
        threshold = bins[min_deriv_idx+1]
        print(f"Steepest slope (min derivative) found at intensity value: {threshold:.4f}")
    else:
        # If peak is at the very left, set threshold to 0
        threshold = 0
        min_deriv_idx = 0
        print("Peak is at the leftmost edge. Setting threshold to 0")

    return threshold, hist, hist_smooth, bins, peak_idx, min_deriv_idx, derivative

# Process each file
for file_path in processed_files:
    filename = os.path.basename(file_path)
    print(f"\nProcessing: {filename}")

    # Load the image
    try:
        img = tifffile.imread(file_path)
        print(f"Image loaded successfully, shape: {img.shape}, dtype: {img.dtype}")
    except Exception as e:
        print(f"Error loading image {filename}: {e}")
        print("Skipping to next file")
        continue

    # Apply illumination correction
    illumination_corrected = apply_illumination_correction(img)

    # Find threshold using the derivative-based method
    threshold, hist, hist_smooth, bins, peak_idx, min_deriv_idx, derivative = find_derivative_threshold(illumination_corrected)

    # Apply thresholding to segment the image
    segmented = np.zeros_like(illumination_corrected)
    segmented[illumination_corrected <= threshold] = 1  # Segment pixels with intensity <= threshold

    # Visualization
    plt.figure(figsize=(15, 12))

    # Plot the original histogram with the detected peak and threshold
    plt.subplot(3, 2, 1)
    plt.bar(bins[:-1], hist, width=bins[1] - bins[0], alpha=0.7)
    plt.plot(bins[:-1], hist_smooth, 'r-', linewidth=2)
    peak_value = bins[peak_idx]
    plt.axvline(x=peak_value, color='g', linestyle='--', label=f'Peak: {peak_value:.4f}')
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    plt.title('Intensity Histogram with Detected Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    # Plot the derivative
    plt.subplot(3, 2, 2)
    # Need to align derivative values with bin centers
    bin_centers = (bins[:-1] + bins[1:]) / 2
    plt.plot(bin_centers[:-1], derivative, 'b-', linewidth=2)
    plt.axvline(x=bins[min_deriv_idx+1], color='r', linestyle='--',
                label=f'Min Derivative: {bins[min_deriv_idx+1]:.4f}')
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.title('First Derivative of Smoothed Histogram')
    plt.xlabel('Intensity Value')
    plt.ylabel('Derivative')
    plt.legend()

    # Display the original image
    plt.subplot(3, 2, 3)
    plt.imshow(illumination_corrected, cmap='gray')
    plt.colorbar(label='Intensity')
    plt.title('Illumination Corrected Image')

    # Display the segmented image
    plt.subplot(3, 2, 4)
    plt.imshow(segmented, cmap='binary')
    plt.title('Segmented Image')

    # Show a comparison of the original and segmented images
    plt.subplot(3, 2, 5)
    # Create an RGB image for overlay
    overlay = np.zeros((*illumination_corrected.shape, 3))
    # Original in grayscale
    for i in range(3):
        overlay[..., i] = illumination_corrected
    # Segmented areas in red
    overlay[segmented == 1, 0] = 1  # Red channel
    overlay[segmented == 1, 1] = 0  # Green channel
    overlay[segmented == 1, 2] = 0  # Blue channel
    plt.imshow(overlay)
    plt.title('Overlay: Segmented Areas in Red')

    # Show a zoomed view of the histogram around the threshold
    plt.subplot(3, 2, 6)
    # Calculate zoom range: 20% of the range centered on the threshold
    zoom_width = (bins[-1] - bins[0]) * 0.2
    zoom_min = max(0, threshold - zoom_width/2)
    zoom_max = min(1, threshold + zoom_width/2)

    # Filter bins within the zoom range
    mask = (bins[:-1] >= zoom_min) & (bins[:-1] <= zoom_max)
    plt.bar(bins[:-1][mask], hist[mask], width=(bins[1] - bins[0]), alpha=0.7)
    plt.plot(bins[:-1][mask], hist_smooth[mask], 'r-', linewidth=2)
    plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
    plt.title('Zoomed View of Histogram Around Threshold')
    plt.xlabel('Intensity Value')
    plt.ylabel('Frequency')
    plt.legend()

    plt.tight_layout()

    # Save the visualization to the images directory
    vis_path = os.path.join(output_img_dir, f"{os.path.splitext(filename)[0]}_visualization.png")
    plt.savefig(vis_path, dpi=300)
    print(f"  Saved visualization to: {vis_path}")
    plt.close()

    # Save the segmented image to the TIFF directory
    seg_path = os.path.join(output_tif_dir, f"{os.path.splitext(filename)[0]}_segmented.tif")
    tifffile.imwrite(seg_path, segmented.astype(np.uint8))
    print(f"  Saved segmented image to: {seg_path}")

print("\nAll processing completed!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 24 TIF files to process

Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1).tif
Image loaded successfully, shape: (1024, 1024), dtype: uint16
Applying illumination correction via division method...
  Using Gaussian filter with sigma = 51
Peak found at intensity value: 0.0586
Steepest slope (min derivative) found at intensity value: 0.0078
  Saved visualization to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes_images/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1)_visualization.png
  Saved segmented image to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20/Holes/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional (1)_segmented.tif

Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Cadherins_regional.tif
Image l